In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Image Processing with Neural Network
## Template 01

## Template with helper functions

- Moons Dataset
- Decision Boundary
- Loss curve

<img src='../../../images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com" align = "left"/>

## Import Statements

In [4]:
###-----------------
### Import Libraries
###-----------------

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import datasets
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
 
#from utils.helper import fn_plot_decision_boundary # this function is from helper.py

In [5]:
#print (fn_plot_decision_boundary.__doc__)

## Setup Global Parameters

In [7]:
###----------------
### Some parameters
###----------------

RANDOM_STATE = 24 # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)
rng = np.random.default_rng(seed = RANDOM_STATE) # Set Random Seed for reproducible  results

NOISE = 0.2
EPOCHS = 200  # number of epochs
ALPHA = 0.1  # learning rate
N_SAMPLES = 1000

# parameters for Matplotlib
params = {'legend.fontsize': 'medium',
          'figure.figsize': (15, 6),
          'axes.labelsize': 'large',
          'axes.titlesize':'large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'
         }

plt.rcParams.update(params)

CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

In [8]:
def fn_plot_decision_boundary(predict,X,wts):
    
    # initialize the figure
    fig, ax = plt.subplots(figsize = (8,5));
    
    # Small increment to create object function surface
    dm = 0.01
    
    # Range of X values (First feature)
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    
    # Range of Y values (Second feature)
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    # Create a grid to evaluate the object function
    xx, yy = np.meshgrid(np.arange(x_min, x_max, dm),
                         np.arange(y_min, y_max, dm))
    
    # flatten and stack
    # arrays will be stacked along their last axis after being upgraded to at least 2-D 
    # with 1’s post-pended to the shape (column vectors made out of 1-D arrays)
    XX = np.c_[xx.ravel(), yy.ravel()] 

    # add another column filled with 1 to make it work with the predict function
    XX = np.hstack( ( XX, np.ones((XX.shape[0],1)) ) )
    
    # list to collect predictions
    y_pred = []
    
    # Iterate over rows
    for row in (XX):
        
        ###change for sigmoid function
        #--------------------------------
        a=predict(row, wts) # this is a float between [0,1]
        y_p =0
        if >=0.5:
            y_p=1

        y_p = predict(row, wts)
        y_pred.append(y_p)
        #--------------------------------------------
    Z = np.array(y_pred).reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.6, cmap=plt.cm.Paired)
    ax.scatter(X[:, 0], X[:, 1], c=X[:, 2],
                                  s=20, edgecolor='k', cmap=plt.cm.bwr)
    ax.set_title('Decision Boundary')

    ax.set_xlabel('A')
    ax.set_ylabel('B')
    plt.show()

SyntaxError: invalid syntax (781074821.py, line 37)

## Generate Data Set
<div style="font-family: Arial; font-size:1.2em;color:black;">
Sklearn's dataset generator is good source of data for learning. To keep the example simple, I'll suggest  <a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html">make_moon</a> dataset generator.
</div>

In [ ]:
X,y=datasets.make_moons(n_samples=N_SAMPLES,
                        noise=NOISE,
                        shuffle=True,
                        random_state= RANDOM_STATE)

X[:5],y[:5]

In [ ]:
data_df= pd.DataFrame(X,columns=['A','B'])
data_df['target']=y
data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df.describe()

In [ ]:
data_df.target.unique()

In [ ]:
fig, ax = plt.subplots() # Instantiate
    
data_df['target'].value_counts().plot(ax=ax, kind='bar');

In [ ]:
ax = sns.scatterplot(data=data_df, 
                     x='A', y='B', 
                     hue='target');

In [ ]:
X, y = datasets.make_moons(n_samples=N_SAMPLES, 
                           shuffle=True, 
                           noise=NOISE, 
                           random_state=RANDOM_STATE)
X[:5], y[:5]

## Visualization
<p style="font-family: Arial; font-size:1.2em;color:black;">
DataFrames are easier to visualize
</p>

In [ ]:
data_df = pd.DataFrame(X, columns = ['A', 'B'])

data_df['target'] = y

data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df.describe() # Generate descriptive statistic

In [ ]:
# what labels are present?
data_df.target.unique()

### Different ways of plotting data

In [ ]:
data_train = data_df[:900].to_numpy()
data_test = data_df[900:].to_numpy()
data_train.shape,data_test.shape

<h4>Perceptron</h4>

In [ ]:
## Perceptron
# z = w0+x1*w1+x2*w2
# z>= 0 a = 1


In [ ]:
def fn_sigmoid(z):
    sig_z = 1.0 / (1 + np.exp(-z))
    return sig_z

In [ ]:
def predict(row,wts):
    z = wts[0]
    for i in range (len(row)-1):
        z+=wts[i + 1] * row[i]
    return fn_sigmoid(z)
   # return 1 if z>=0 else 0

In [ ]:
def train_weights(train,alpha,n_epochs):
    errors =[] #store all errors
    weights =rng.random(train.shape[1]) # weights initialized at random
    m = train.shape[0] #number of samples
    for epoch in np.arange(n_epochs): # looping over all epochs
        sum_error = 0.0

        for row in train:
            prediction= predict(row,weights)

            dz= prediction - row[-1]#(a-y)
            sum_error+=(prediction - row[-1])**2 # mean square error
            #cross entropy loss = y.log(a)
            #sum_error +=-row[-1]*log(prediction)
            weights[0]=weights[0]-alpha*dz/m # update w0

            for i in range (len(row)-1):
                weights[i+1]+=-alpha*dz*row[i]/m # update w1 and w2

        errors.append(sum_error/m)

    return weights,errors

In [ ]:
weights,errors = train_weights(data_train,ALPHA,EPOCHS)

In [ ]:
#weights,error=train_weights(data_train,ALPHA,EPOCHS)

In [ ]:
predictions =[]
for row in data_train:
         prediction = predict(row , weights)
         predictions.append(predictions)
    
accuracy_score(y_pred = predictions, y_true=data_train[:,-1])




In [ ]:
fn_plot_decision_boundary(predict,data_train,weights)

In [ ]:
cm = confusion_matrix(y_true=data_test[:,-1],y_pred=predictions)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [0,1])

disp.plot(cmap='Blues',colorbar=False)
plt.grid()
plt.show()

In [ ]:
predictions =[] # to store %predictons
for row in data_test: # iterate over rows
    prediction = predict(row , weights) # make predictions
    pred = 0
    if(prediction >= 0.5):
        pred = 1
    predictions.append(pred)  # collect in a list

#calculate accuracy
print(f'Test Accuracy:{accuracy_score(y_pred = predictions, y_true=data_test[:,-1])}')

#calculate confusion
cm= confusion_matrix(y_true=data_test[:,-1],y_pred=predictions)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [0,1])

disp.plot(cmap='Blues',colorbar=False)
plt.grid()
plt.show()


In [ ]:
fn_plot_decision_boundary(predict,data_train,weights)